In [13]:
import glob
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import numpy as np
import IPython.display as ipd
import re
import jamotools
import os

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import librosa

In [14]:
data_path = 'D:/aihub/ETRI_metadata.npy'

In [15]:
metadata = np.load(data_path)

In [16]:
n_mels = 80
fs = 16000
frame_length_ms = 50
frame_shift_ms = 25
nsc = int(fs * frame_length_ms / 1000)
nov = nsc - int(fs * frame_shift_ms / 1000)
nhop = int(fs * frame_shift_ms / 1000)
eps = 1e-8
db_ref = 160

In [23]:
print(len(metadata))

for i, data in tqdm(enumerate(metadata)):
    pcm_path = data[0]
    txt_path = data[1]
    
    mel_dir = '\\'.join(pcm_path.replace('aihub\\', 'aihub\\mel\\').split('\\')[:-1])
    mel_path = pcm_path.replace('aihub\\', 'aihub\\mel\\').replace('.pcm', '.npy')
    
    if not os.path.exists(mel_dir):
        os.makedirs(mel_dir)
    
    if not os.path.isfile(mel_path):
        
        try:
            with open(pcm_path, 'rb') as pcm_file:
                pcm_data = np.fromfile(pcm_file, dtype=np.int16)
                y = pcm_data / 2 ** 14

                f, t, Zxx = sp.signal.stft(y, fs=fs, nperseg=nsc, noverlap=nov)

                Sxx = np.abs(Zxx)

                mel_filters = librosa.filters.mel(sr=fs, n_fft=nsc, n_mels=n_mels)
                mel_specgram = np.matmul(mel_filters, Sxx)

                log_mel_specgram = 20 * np.log10(np.maximum(mel_specgram, eps))
                norm_log_mel_specgram = (log_mel_specgram + db_ref) / db_ref

                np.save(mel_path, norm_log_mel_specgram)

#             plt.figure(figsize=(10,10))
#             plt.imshow(norm_log_mel_specgram, origin='lower')
#             plt.colorbar()
#             plt.show()

        except:
            print("Error found in {}".format(pcm_path))

620799
